In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import matplotlib.pyplot as plt

In [2]:
list1 = [1,2,3,4,5,6,77,88,99,23,42,13,55,4,4,67]
print(sorted(list1))
print(f"The list has {len(list1)} numbers")

def central_tendancy(x):
    return print(f"Sum: {sum(x)}\nMin: {min(x)}\nMax: {max(x)}\nMean: {sum(x)/len(x):.3f}\nMedian: {np.median(x)}\nRange: {max(x)-min(x)}")

central_tendancy(list1)

[1, 2, 3, 4, 4, 4, 5, 6, 13, 23, 42, 55, 67, 77, 88, 99]
The list has 16 numbers
Sum: 493
Min: 1
Max: 99
Mean: 30.812
Median: 9.5
Range: 98


In [3]:
df = pd.read_csv('../../data/raw/listings.csv')
print(df.head(5))
print('='*50)
print(df.info())
print('='*50)
print(df.describe())

     id                        listing_url       scrape_id last_scraped  \
0  5456  https://www.airbnb.com/rooms/5456  20250613040113   2025-06-13   
1  5769  https://www.airbnb.com/rooms/5769  20250613040113   2025-06-13   
2  6413  https://www.airbnb.com/rooms/6413  20250613040113   2025-06-14   
3  6448  https://www.airbnb.com/rooms/6448  20250613040113   2025-06-13   
4  8502  https://www.airbnb.com/rooms/8502  20250613040113   2025-06-13   

            source                                               name  \
0      city scrape          Walk to 6th, Rainey St and Convention Ctr   
1      city scrape                                     NW Austin Room   
2  previous scrape                      Gem of a Studio near Downtown   
3      city scrape  Secluded Studio @ Zilker - King Bed, Bright & ...   
4      city scrape                            Woodland Studio Lodging   

                                         description  \
0  Great central  location for walking to Convent...  

In [4]:
# clean price column
df['price'] = df['price'].str.replace(r'[$,]','',regex=True).astype(float)
df['price'].describe()

count    10708.000000
mean       386.470583
std       2620.198322
min          9.000000
25%         89.000000
50%        138.000000
75%        235.000000
max      50000.000000
Name: price, dtype: float64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15187 entries, 0 to 15186
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            15187 non-null  int64  
 1   listing_url                                   15187 non-null  object 
 2   scrape_id                                     15187 non-null  int64  
 3   last_scraped                                  15187 non-null  object 
 4   source                                        15187 non-null  object 
 5   name                                          15187 non-null  object 
 6   description                                   14840 non-null  object 
 7   neighborhood_overview                         7914 non-null   object 
 8   picture_url                                   15186 non-null  object 
 9   host_id                                       15187 non-null 

In [6]:
df[df['price']<750].groupby('listing_url')[[
    'estimated_occupancy_l365d','estimated_revenue_l365d']].max().sort_values(
    'estimated_revenue_l365d',ascending=False).head(15)

,estimated_occupancy_l365d,estimated_revenue_l365d
listing_url,,
https://www.airbnb.com/rooms/1256785132829355369,255,187170.0
https://www.airbnb.com/rooms/547037499443380880,240,169920.0
https://www.airbnb.com/rooms/1103570058000753689,255,162690.0
https://www.airbnb.com/rooms/1034416942292085819,255,158865.0
https://www.airbnb.com/rooms/1248863625055330289,222,156066.0
https://www.airbnb.com/rooms/1155778073697659775,255,151725.0
https://www.airbnb.com/rooms/929056006339989028,255,147390.0
https://www.airbnb.com/rooms/45432352,255,146625.0
https://www.airbnb.com/rooms/1198768133864182591,198,139788.0


In [7]:
# same return using SQL
conn = sql.connect(':memory:')
df.to_sql('austin_housing', conn, index=False)

query = """ SELECT listing_url,
    MAX(estimated_occupancy_l365d) as days_booked_annually,
    MAX(estimated_revenue_l365d) as annual_income
    FROM austin_housing
    WHERE price < 750
    GROUP BY listing_url
    ORDER BY annual_income DESC
"""

result = pd.read_sql(query, conn)
print(result.head(5))

conn.close()

                                        listing_url  days_booked_annually  \
0  https://www.airbnb.com/rooms/1256785132829355369                   255   
1   https://www.airbnb.com/rooms/547037499443380880                   240   
2  https://www.airbnb.com/rooms/1103570058000753689                   255   
3  https://www.airbnb.com/rooms/1034416942292085819                   255   
4  https://www.airbnb.com/rooms/1248863625055330289                   222   

   annual_income  
0       187170.0  
1       169920.0  
2       162690.0  
3       158865.0  
4       156066.0  
